### User Interface with Gradio for QG & QA
T5 Answer Agnostic

In [8]:
import gradio as gr
from transformers import pipeline, BartForConditionalGeneration, AutoTokenizer
import Levenshtein
import os
import markdown
from bs4 import BeautifulSoup
import pickle
from utils import Chapter, File

# SAVE_TREE_DIR = os.path.join("C:", os.sep, "Users", "ManuV", "Documents", "Bachelorproef", "BP_Info_Support", "data")
# SAVE_TREE_DIR = os.path.join("E:", os.sep, "HOGENT", "2022_2023", "BA","BP_Info_Support","data")
# C:\Users\dama_\OneDrive\Desktop\HoGent\2022_2023\BP\BP_Info_Support
SAVE_TREE_DIR = os.path.join("C:", os.sep, "Users", "dama_", "OneDrive","Desktop","HoGent","2022_2023","BP","BP_Info_Support","data")

QG_MODEL_NAME = "ManuVleuBeu/T5_ag_SQuAD"
QA_MODEL_NAME = "damapika/roberta-base_mod_squad"

In [9]:
def get_guidance_framework():
    with open(os.path.join(SAVE_TREE_DIR,"gf_structure.pkl"), "rb") as f:
        return pickle.load(f)
    
root = get_guidance_framework()
    
def get_all_files():
    chapters = root.get_all_chapters()

    all_files = []
    for chapter in chapters:
        all_files.extend(chapter.files)

    return all_files

files = get_all_files()

def clean_file_dirs():
    for file in files:
        file_dir = '/'.join(file.dir.split('docs\\')[1:])
        file.dir = file_dir.replace('\\','/')

    return files

files = clean_file_dirs()

def get_contexts_from_file_dir(file_dir):
    file = [file for file in files if file.dir == file_dir]
    if not len(file) == 0:
        file = file[0]
        # Sorts text per h-elem by biggest
        key_ranking = sorted(file.text.keys(), key=lambda k: len(' '.join(file.text[k])), reverse=True)
        return [' '.join(file.text[key]) for key in key_ranking]
    else:
        raise Exception("File directory does not exist!")


In [20]:
files = get_all_files()

# Question Generation Model
question_generator = pipeline("text2text-generation", model=QG_MODEL_NAME)
# question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qa-qg-hl")

# Answer Generation Model
answer_generator = pipeline("question-answering", model=QA_MODEL_NAME)
# TODO: Add all chapter contexts or an extra function where user selects chapter->context
contexts = [file.dir for file in files]
selected_context = []
generated_question = []

def calculate_similarity(answer1, answer2):
    similarity_score = 1 - (Levenshtein.distance(answer1.lower(), answer2.lower()) / max(len(answer1), len(answer2)))
    return similarity_score

def generate_question(file_dir):
    global selected_context
    global generated_question
    contexts = get_contexts_from_file_dir(file_dir)
    selected_context.clear()
    generated_question.clear()
    for context in contexts:
        question = question_generator(context)[0]['generated_text']
        question = question.split('?')[0]+'?'# Selects first question
        # questions = [question.strip() for question in string.split('?') if question.endswith('?')] instead of first question takes multiple per context
        generated_question.append(question)
    selected_context = contexts
    return '\n'.join(generated_question)

# Interface for QG
io1 = gr.Interface(
    fn=generate_question,
    inputs=[
        gr.inputs.Dropdown(choices=contexts, label="Select a context", type="value"),
    ],
    live=False,
    outputs=[
        gr.outputs.Textbox(label="Generated Question"),
    ],
    title="Quiz Generator",
    description="Select a context, generate a question, and compare your answer.",
    allow_flagging="never",
)

def generate_answer(question,user_answer):
    context = selected_context[generated_question.index(question)]
    answer = answer_generator(context=context,question=question)
    score=calculate_similarity(user_answer,answer['answer'])
    return [user_answer,answer['answer'],score]
io1.launch()


c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\dama_\AppData\Local\Temp\ipykernel_14204\4064746390.py:54: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'interface_id': 'io2'}
  io2 = gr.Interface(


Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Additional interface for answer checking
io2 = gr.Interface(
    fn=generate_answer,
    live=False,
    inputs=[gr.inputs.Dropdown(choices=generated_question,label="Select a question",type="value"), gr.outputs.Textbox(label="User Answer")],
    outputs=[gr.outputs.Textbox(label="User Answer"),gr.outputs.Textbox(label="Generated Answer"),gr.outputs.Textbox(label="Levenshtein Answer score")],
    title="Answer Checker",
    description="Compare your answer.",
    allow_flagging="never",
    interface_id = "io2"
)

io2.launch()

BART_ans_SQuAD

In [1]:
import gradio as gr
from transformers import pipeline, BartForConditionalGeneration, AutoTokenizer
import Levenshtein
import os
import markdown
from bs4 import BeautifulSoup
import pickle
from utils import Chapter, File
import spacy
import textrazor
import random

SAVE_TREE_DIR = os.path.join("C:", os.sep, "Users", "ManuV", "Documents", "Bachelorproef", "BP_Info_Support", "data")
QG_MODEL_NAME = "ManuVleuBeu/BART_ans_SQuAD"
QA_MODEL_NAME = "damapika/roberta-base_mod_squad"
TEXTRAZOR_APIKEY = "42acc4018111e29f626f458272bd34981a5b2b268c5fc8dbcacd7ab5"

c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_guidance_framework():
    with open(os.path.join(SAVE_TREE_DIR,"gf_structure.pkl"), "rb") as f:
        return pickle.load(f)
    
root = get_guidance_framework()
    
def get_all_files():
    chapters = root.get_all_chapters()

    all_files = []
    for chapter in chapters:
        all_files.extend(chapter.files)

    return all_files

files = get_all_files()

def clean_file_dirs():
    for file in files:
        file_dir = '/'.join(file.dir.split('docs\\')[1:])
        file.dir = file_dir.replace('\\','/')

    return files

files = clean_file_dirs()

def get_contexts_from_file_dir(file_dir):
    file = [file for file in files if file.dir == file_dir]
    if not len(file) == 0:
        file = file[0]
        # Sorts text per h-elem by biggest
        key_ranking = sorted(file.text.keys(), key=lambda k: len(' '.join(file.text[k])), reverse=True)
        return [' '.join(file.text[key]) for key in key_ranking]
    else:
        raise Exception("File directory does not exist!")
    
def run_model(input_string,target_answer,tokenizer,model, **generator_args):
    generator_args = {
    "max_length": 768,
    "num_beams": 4,# bij grotere num_beams is trager maar complexere vragen(niet per se betere vragen)
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    
    input_string = input_string + " " + "<ANSWER>" + target_answer + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids.to("cuda:0"), **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    output = [item.split("<sep>") for item in output]
    return output[0][0]

def get_marked_answers(context):
    marked_answers = []

    # TextRank
    # If error -> python -m spacy download en_core_web_sm
    # nlp = spacy.load("en_core_web_sm")
    # nlp.add_pipe("textrank")
    # doc = nlp(context)

    # for phrase in doc._.phrases[:2]:
    #     marked_answers.append(phrase.text.replace('\n',''))

    # TextRazor
    textrazor.api_key = TEXTRAZOR_APIKEY
    client = textrazor.TextRazor(extractors=["entities","topics"])
    response = client.analyze(context)

    textrazor_results = {"word": [], "score": []}
    for entity in response.entities():
        textrazor_results["score"].append(entity.relevance_score)
        textrazor_results["word"].append(entity.id)

    # Sort the words based on scores in descending order
    sorted_words = sorted(textrazor_results["word"], key=lambda w: textrazor_results["score"][textrazor_results["word"].index(w)], reverse=True)

    # Get the top 3 words
    top_words = sorted_words[:3]

    marked_answers.extend(top_words)

    marked_answers = list(set(marked_answers))
    # random.shuffle(marked_answers)

    return marked_answers  


In [3]:
files = get_all_files()

# Question Generation Model
qg_model = BartForConditionalGeneration.from_pretrained(QG_MODEL_NAME).to("cuda:0")
qg_tokenizer = AutoTokenizer.from_pretrained(QG_MODEL_NAME)

# Answer Generation Model
answer_generator = pipeline("question-answering", model=QA_MODEL_NAME)
# TODO: Add all chapter contexts or an extra function where user selects chapter->context
contexts = [file.dir for file in files]
selected_context = []
generated_question = []
selected_answers = []

def calculate_similarity(answer1, answer2):
    similarity_score = 1 - (Levenshtein.distance(answer1.lower(), answer2.lower()) / max(len(answer1), len(answer2)))
    return similarity_score

def generate_question(file_dir):
    global selected_context
    global generated_question
    global selected_answers
    contexts = get_contexts_from_file_dir(file_dir)
    selected_context.clear()
    generated_question.clear()
    
    for context in contexts:
        answer = get_marked_answers(context)[0]
        selected_answers.append(answer)
        question = run_model(context,answer,qg_tokenizer,qg_model)
        question = question.split('?')[0]+'?'# Selects first question
        # questions = [question.strip() for question in string.split('?') if question.endswith('?')] instead of first question takes multiple per context
        generated_question.append(question)
    selected_context = contexts
    return '\n'.join(generated_question)

# Interface for QG
io1 = gr.Interface(
    fn=generate_question,
    inputs=[
        gr.inputs.Dropdown(choices=contexts, label="Select a context", type="value"),
    ],
    live=False,
    outputs=[
        gr.outputs.Textbox(label="Generated Question"),
    ],
    title="Quiz Generator",
    description="Select a context, generate a question, and compare your answer.",
    allow_flagging="never",
)

c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


In [4]:
io1.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [11]:
def generate_answer(question,user_answer):
     #context = selected_context[generated_question.index(question)]
    answer = selected_answers[generated_question.index(question)]
    score=calculate_similarity(user_answer,answer)
    return [user_answer,answer,score]

# Additional interface for answer checking
io2 = gr.Interface(
    fn=generate_answer,
    live=False,
    inputs=[gr.inputs.Dropdown(choices=generated_question,label="Select a question",type="value"), gr.outputs.Textbox(label="User Answer")],
    outputs=[gr.outputs.Textbox(label="User Answer"),gr.outputs.Textbox(label="Generated Answer"),gr.outputs.Textbox(label="Levenshtein Answer score")],
    title="Answer Checker",
    description="Compare your answer.",
    allow_flagging="never",
)

io2.launch()

c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\ManuV\AppData\Local\Temp\ipykernel_12172\1161264703.py:8: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'interface_id': 'io2'}
  io2 = gr.Interface(


Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.
